In [2]:
from city_sustainability.models import unet_model

2023-05-24 17:51:42.925038: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 17:51:43.081816: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-05-24 17:51:43.082967: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-24 17:51:44.325113: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
import PIL

# import os
from PIL import Image

def image_resize(width, height, image):
    # os.makedirs(out_dir, exist_ok=True)
    # new_image = Image.open(image_file)
    resized_image = image.resize((width, height),resample=Image.Resampling.NEAREST)
    return resized_image

In [4]:
import os

def loading_paths(data_path):
    # data_path is the path where all the city folders are located
    city_folders = os.listdir(data_path)

    # Initialize empty lists for images and labels
    image_paths = []
    label_paths = []

    # Iterate over city folders
    for city_folder in city_folders:
        city_path = os.path.join(data_path, city_folder)
        # Check if the item is a directory
        if os.path.isdir(city_path):
            image_folder = os.path.join(city_path, "images")
            label_folder = os.path.join(city_path, "labels")
            # Check if image and label folders exist
            if os.path.exists(image_folder) and os.path.exists(label_folder):
                # List all image files
                image_files = os.listdir(image_folder)
                # Iterate over image files
                for image_file in image_files:
                    image_path = os.path.join(image_folder, image_file)
                    label_path = os.path.join(label_folder, image_file)
                    if os.path.exists(label_path):
                        # Append image and label file paths to lists
                        image_paths.append(image_path)
                        label_paths.append(label_path)

    return image_paths, label_paths


import numpy as np
from PIL import Image
from tensorflow.keras.utils import to_categorical
from city_sustainability.preprocessing import image_resize

def image_and_label_arrays(image_paths, label_paths, sampling_ratio = 1):
    # Default sampling_ratio is equal to 1.0 (selects all samples)
    image_list_array = []
    label_list_array = []
    end = round(sampling_ratio*len(image_paths))

    for image_path in image_paths[0:end]:
        im = Image.open(image_path)
        # Resize each image using image_resize function
        resized_image = image_resize(256,256,im)
        # Generate array for each image
        numpy_array_image = np.array(resized_image)
        # Add resized array to list
        image_list_array.append(numpy_array_image)

    for label_path in label_paths[0:end]:
        lb = Image.open(label_path)
        # Generate array for each label
        # Resize each label using image_resize function
        resized_label = image_resize(256,256,lb)
        # Generate array for each image
        numpy_array_label = np.array(resized_label)
        # Encode labels
        encoded_label = to_categorical(numpy_array_label, num_classes=9)
        # Add resized and encoded array to list
        label_list_array.append(encoded_label)

    from sklearn.metrics import accuracy_score

# Assuming you have the ground truth labels and the total number of instances


    image_array = np.array(image_list_array) # Convert the list of arrays to a NumPy array
    label_array = np.array(label_list_array)  # Convert the list of arrays to a NumPy array

    return image_array, label_array


In [5]:
data_path_1 = '../../raw_data/OpenEarthMap_wo_xBD'
image_paths, label_paths = loading_paths(data_path_1)

In [6]:
X, y = image_and_label_arrays(image_paths, label_paths, 0.01)

In [7]:
model = unet_model.build_vgg16_model(input_shape=(256,256, 3), num_classes=9)

### Compile the model:-)
unet_model.compile_model(model)

### Train the model:-)
unet_model.train_model(model, X , y, epochs=1, batch_size=512, validation_split=0.2)

2023-05-24 17:51:56.148801: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-05-24 17:51:56.149156: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


1/1 [==============================] - 9s 9s/step - loss: 2.6310 - accuracy: 0.1318 - val_loss: 17.4271 - val_accuracy: 0.3498 - lr: 1.0000e-04
